## 🔴 **SAR Wizards: Unleash your radar magic** 🧙‍♂️✨  

You've been working so hard in the previous exercises—now it's time to have some simulation fun! 🚀  

---

### **🔹 Task 1: Simulate Any Targets You Want**  

We’ve provided code to **simulate echo signals** for a **phased array system**. Your job? **Choose your own target locations** and see how they appear in radar images!  

🔧 **How it works:**  
- Define your targets in **(x, y) coordinates**.  
- The simulation will generate the **echo signals** for your chosen targets.  
- Use the **beamforming code** you completed in the Maverick exercise to process the **range-azimuth radar images**.  
- Experiment with **different target configurations** and see how they affect the image!  

🎯 Can you arrange targets to create a recognizable shape? Take into account that the **native radar coordinates are range and azimuth (i.e., polar coordinates)**, whereas you are defining the targets in Cartesian coordinates.

---

### **🔹 Task 2: Identify an Approximation in the Simulation**  

If you’re a **true SAR Wizard**, you might have already sensed that **something isn't quite right** in our phased array simulation.  

🕵️ **Investigate `generate_target_echoes()`** – we took a bit of a **shortcut** and used a **simplifying approximation** to make things easier.  

🔍 **Your mission:**  
1. **Find the approximation.** What assumption(s) did we make?  
2. **Analyze its limitations.** When does it hold, and when does it break down?  

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
c = 299792458  # Speed of light (m/s)

In [ ]:
# Function to simulate target echoes in range for a number of array elements
def generate_target_echoes(pulse_duration, target_ranges, target_az_angles, sampling_rate, total_time, amplitudes, wavelength, array_size):
    """
    Simulates the received radar pulses for each element in an antenna array, 
    incorporating both range delays and phase shifts due to angle.
    """
    num_samples = int(total_time * sampling_rate)
    t = np.linspace(0, total_time, num_samples, endpoint=False)
    k = 2 * np.pi / wavelength
    
    # Define the antenna array element positions (Uniform Linear Array)
    d = wavelength / 2  # Element spacing (half-wavelength spacing)
    positions = np.linspace(-(array_size-1)/2 * d, (array_size-1)/2 * d, array_size)

    # Initialize signal array (each row is an antenna element)
    received_signals = np.zeros((array_size, num_samples), dtype=np.complex64)

    for i, (target_range, target_angle) in enumerate(zip(target_ranges, target_az_angles)):
        # Compute time delay for round-trip echo
        round_trip_time = 2 * target_range / c  # Time delay for the pulse to reach & return

        # Compute index range for pulse in time vector
        pulse_start_idx = int(round_trip_time * sampling_rate)
        pulse_end_idx = pulse_start_idx + int(pulse_duration * sampling_rate)

        # Compute phase shift for each array element
        target_phasor = np.exp(1j * 2 * np.pi * 2 * target_range / wavelength)  # Phase from range
        phase_shifts = np.exp(1j * k * positions * np.sin(target_angle))  # Phase shift per array element due to target angle
        
        # Ensure pulse fits within signal array
        if pulse_end_idx < num_samples:
            pulse = np.ones(pulse_end_idx - pulse_start_idx)  # Simple rectangular pulse
            
            # Apply phase shifts for each antenna element and target phase due to range
            for e in range(array_size):
                received_signals[e, pulse_start_idx:pulse_end_idx] += (
                    pulse * amplitudes[i] * target_phasor * phase_shifts[e]
                )

    return t, received_signals

In [ ]:
# Function to combine the received element echoes into an angular profile
def compute_beam_output(element_phasors, scan_angles_rad, positions, wavelength):
    """
    Perform an angular scan using the received element signals, to obtain the received wave height vs. angle
    """
    beam_output = np.zeros((len(scan_angles_rad),), dtype=np.complex64)

    for i, scan_angle in enumerate(scan_angles_rad):
        # Element phase shifts
        scan_phase_shift = 2 * np.pi * positions * np.sin(scan_angles_rad[i]) / wavelength
        scan_phase_shifts = np.exp(-1j * scan_phase_shift) # Phase correction corresponding to this scan angle
        beam_output[i] = np.sum(element_phasors * scan_phase_shifts) # apply phase correction and sum phasors together

    return scan_angles_rad, beam_output

### Now it's time to define the simulation parameters! Feel free to adjust these as you wish! 🔬 

In [ ]:
# User-defined parameters
range_resolution = 40  # m
pulse_duration = 2 * range_resolution / c  # pulse duration (s)
sampling_rate = 5 / pulse_duration  # 5 samples per pulse
wavelength = 0.03  # Radar wavelength (3 cm)
number_of_elements = 50  # Number of antenna elements

### Generate your own values for the target (x,y)-coordinates! 🧪

In [ ]:
# Generate target positions, assuming the radar is at the origin
x_targets = np.array([100, -100]) # x-axis is along the array dimension
y_targets = np.array([1000,1500]) # y-axis is perpendicular to the array

# Plot the target locations
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(x_targets, y_targets, color='red', s=50, marker="o", label="Targets")
ax.scatter(0, 0, color='black', marker='x', s=100, label="Radar Position")  # Radar at origin

# Formatting
ax.set_xlabel("X Position (m)")
ax.set_ylabel("Y Position (m)")
ax.set_title("Target Distribution")
ax.legend()
ax.grid(True, linestyle="--", linewidth=0.5)

### Then we calculate target range and azimuth and simulate the echoes 🌊

In [ ]:
# Calculate target ranges and angles based on their (x,y) locations
target_ranges = []
target_angles = []
for i in range(len(x_targets)):
    range_tgt = np.sqrt(x_targets[i]**2 + y_targets[i]**2)
    angle_tgt = np.arctan2(x_targets[i], y_targets[i])
    target_ranges.append(range_tgt)
    target_angles.append(angle_tgt)

# Define recording window length based on target ranges
max_range = np.max(target_ranges)
total_time = 2 * max_range / c + 2 * pulse_duration  # Ensure enough time for full echoes from each target
target_amplitudes = np.ones_like(target_ranges) # unit amplitude for all targets

# Simulate the target echoes
t, received_signals = generate_target_echoes(
    pulse_duration, target_ranges, target_angles, sampling_rate, total_time, target_amplitudes, wavelength, number_of_elements
)


### Let's plot the simulated echo data per element

In [ ]:
plt.figure(figsize=(10, 6))
extent = [0, total_time * c / 2, 0, number_of_elements]  # Time mapped to range
plt.imshow(np.abs(received_signals), aspect='auto', extent=extent, cmap='jet', origin='lower')
plt.colorbar(label="Normalized Intensity")
plt.xlabel("Range (m)")
plt.ylabel("Element #")
plt.title("Received signal per element")
plt.show()

### Then we define our image azimuth axis (scan angles)

In [ ]:
# Compute beamformed response
d = wavelength / 2
element_positions_x = np.linspace(-(number_of_elements-1)/2 * d, (number_of_elements-1)/2 * d, number_of_elements)

# Let's first prepare the azimuth angle axis of the image
angle_min_deg = -50  # Minimum scan angle (degrees)
angle_max_deg = 50   # Maximum scan angle (degrees)

# Determine azimuth angle spacings and the scan angles for the beamforming
# Calculate the width of the array
array_size = (number_of_elements-1) * d # Width of the array (m)
angle_resolution = wavelength / array_size # Approximate formula for angle resolution in radians
# Calculate the angle step based on the resolution
angle_step_deg = angle_resolution / 4  # Angle spacing (degrees), let's oversample by a factor of 2-4 to get a nice image
number_of_angles = int( np.round((angle_max_deg - angle_min_deg) / angle_step_deg) ) # number of azimuth angle pixels in our image
number_of_ranges = len(t) # number of range samples in the data and in our image
ranges = c * t / 2 # range axis
scan_angles_radians = np.linspace( np.radians(angle_min_deg), np.radians(angle_max_deg), number_of_angles ) # azimuth angles

### Now it's time to apply your beamforming code! Reuse your code from the Maverick exercise! 🚀

In [ ]:
# Prepare empty image array
radar_image_2d = np.zeros((number_of_angles, number_of_ranges), dtype=np.complex64)

# Complete the for loop to obtain your first radar image!
# for i in range(number_of_ranges):

## And plot the image
plt.figure(figsize=(10, 6))
extent = [ranges[0], ranges[-1], angle_min_deg, angle_max_deg]  # Time mapped to range
plt.imshow(10*np.log10(np.abs(radar_image_2d)), aspect='auto', extent=extent, cmap='jet', origin='lower')
plt.colorbar(label="Normalized Intensity (dB)")
plt.xlabel("Range (m)")
plt.ylabel("Azimuth angle (deg)")
plt.title("Range-azimuth image")
plt.show()

### Now it's time to analyze the simulation code? What shortcuts are we taking? 🤔